In [87]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import pyabc 
from datetime import timedelta
# Set seed
np.random.seed(42)

# Simulation parameters
N_AUDIENCES = 10  # Number of unique audience members
OBS_PER_AUDIENCE = 8  # Observations per audience member

# True hyperparameters
hyper_mu_intercept = 2.0    # Population-level intercept
hyper_sigma_intercept = 0.5 # Between-audience intercept variability
hyper_mu_slope = -0.03      # Population-level cost slope 
hyper_sigma_slope = 0.01    # Between-audience slope variability

# Simulate audience-specific parameters
audience_ids = np.repeat(np.arange(N_AUDIENCES), OBS_PER_AUDIENCE)
true_intercepts = np.random.normal(hyper_mu_intercept, hyper_sigma_intercept, N_AUDIENCES)
true_slopes = np.random.normal(hyper_mu_slope, hyper_sigma_slope, N_AUDIENCES)

# Simulate cost and clicks
cost = np.random.gamma(shape=5, scale=1, size=N_AUDIENCES*OBS_PER_AUDIENCE)
eta = true_intercepts[audience_ids] + true_slopes[audience_ids] * cost
clicks = np.random.poisson(lam=np.exp(eta))

# Create DataFrame
data = pd.DataFrame({
    "audience_id": audience_ids,
    "cost": cost,
    "clicks": clicks
})
data

,audience_id,cost,clicks
0,0,8.602851,5
1,0,4.195728,3
2,0,3.586684,12
3,0,4.910412,11
4,0,3.485679,8
...,...,...,...
75,9,5.076993,7
76,9,5.250892,7
77,9,6.567539,6
78,9,5.887435,12


In [10]:
distributions = dict(
    mu_intercept = pyabc.RV("norm", 0, 2),
    sigma_intercept = pyabc.RV("halfnorm", 0, 0.5),
    mu_slope = pyabc.RV("norm", 0, 0.1),
    sigma_slope = pyabc.RV("halfnorm", 0, 0.5), 
)

for i in range(N_AUDIENCES):
    distributions[f"alpha_z_{i}"] = pyabc.RV("norm", 0, 1)
    distributions[f"beta_z{i}"] = pyabc.RV("norm", 0, 1)

priors = pyabc.Distribution(**distributions)
priors

<Distribution
    mu_intercept=<RV name=norm, args=(0, 2), kwargs={}>,
    sigma_intercept=<RV name=halfnorm, args=(0, 0.5), kwargs={}>,
    mu_slope=<RV name=norm, args=(0, 0.1), kwargs={}>,
    sigma_slope=<RV name=halfnorm, args=(0, 0.5), kwargs={}>,
    alpha_z_0=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z0=<RV name=norm, args=(0, 1), kwargs={}>,
    alpha_z_1=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z1=<RV name=norm, args=(0, 1), kwargs={}>,
    alpha_z_2=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z2=<RV name=norm, args=(0, 1), kwargs={}>,
    alpha_z_3=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z3=<RV name=norm, args=(0, 1), kwargs={}>,
    alpha_z_4=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z4=<RV name=norm, args=(0, 1), kwargs={}>,
    alpha_z_5=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z5=<RV name=norm, args=(0, 1), kwargs={}>,
    alpha_z_6=<RV name=norm, args=(0, 1), kwargs={}>,
    beta_z6=<RV name=norm, args=(0, 1), kwargs={}>,
    

In [79]:
from sklearn.metrics import mean_absolute_error
from scipy.stats import wasserstein_distance

def distance(x, x0):
    return wasserstein_distance(x["observed"], x["simulated"]) 

def simulator_model(parameters):
    audience_ids = data.audience_id
    cost = data.cost
    observed_clicks = data.clicks.values
    
    alpha_z = []
    beta_z = []
    for i in range(N_AUDIENCES):
        alpha_z.append(parameters[f"alpha_z_{i}"])
        beta_z.append(parameters[f"beta_z{i}"])
    alpha_z = np.array(alpha_z)
    beta_z = np.array(beta_z)
    
    alpha = parameters["mu_intercept"] + alpha_z[audience_ids] * parameters["sigma_intercept"]
    beta = parameters["mu_slope"] + beta_z[audience_ids] * parameters["sigma_slope"]

    eta = alpha[audience_ids] + beta[audience_ids]*cost
    eta = eta.astype(np.float64)
    simulated_clicks = np.random.poisson(lam=np.exp(eta).astype(np.float64))

    observed_vector = [
        np.median(observed_clicks), np.mean(observed_clicks),
        np.std(observed_clicks), np.var(observed_clicks),
        np.quantile(observed_clicks, 0.1), np.quantile(observed_clicks, 0.9),
    ]
    simulated_vector = [
        np.median(simulated_clicks), np.mean(simulated_clicks),
        np.std(simulated_clicks), np.var(simulated_clicks),
        np.quantile(simulated_clicks, 0.1), np.quantile(simulated_clicks, 0.9),
    ]
    
    return {
        "observed": observed_vector, 
        "simulated": simulated_vector
    }

sampler = pyabc.ABCSMC(
    simulator_model,
    priors,
    distance,
    #eps=pyabc.MedianEpsilon(),
    sampler=pyabc.SingleCoreSampler(check_max_eval=True),
)
sampler.new("sqlite://")

history = sampler.run(
    minimum_epsilon=0.5,
    max_walltime=timedelta(minutes=180),
)

ABC.History INFO: Start <ABCSMC id=1, start_time=2025-04-10 20:08:43>
ABC INFO: Calibration sample t = -1.
ABC INFO: t: 0, eps: 9.16749059e+00.
ABC INFO: Accepted: 100 / 187 = 5.3476e-01, ESS: 1.0000e+02.
ABC INFO: t: 1, eps: 7.58674993e+00.
ABC INFO: Accepted: 100 / 353 = 2.8329e-01, ESS: 1.5837e+00.
ABC INFO: t: 2, eps: 2.93261098e+00.
ABC INFO: Accepted: 100 / 1035 = 9.6618e-02, ESS: 2.2670e+00.
ABC INFO: t: 3, eps: 1.82850906e+00.
ABC.Population WARNING: The total population weight 1.6419362823002242e-20 is close to zero, which can be numerically problematic
ABC INFO: Accepted: 100 / 1159 = 8.6281e-02, ESS: 2.4638e+00.
ABC INFO: t: 4, eps: 5.86482893e-01.
ABC.Population WARNING: The total population weight 3.419219528397951e-28 is close to zero, which can be numerically problematic
ABC INFO: Accepted: 100 / 4377 = 2.2847e-02, ESS: 1.0835e+00.
ABC INFO: t: 5, eps: 4.90319253e-01.
ABC.Population WARNING: The total population weight 6.703168379544295e-38 is close to zero, which can be

In [80]:
for plot_func in [
    #pyabc.visualization.plot_sample_numbers_trajectory,
    #pyabc.visualization.plot_epsilons,
    #pyabc.visualization.plot_effective_sample_sizes,
    #pyabc.visualization.plot_walltime,
    #pyabc.visualization.plot_total_walltime,
    #pyabc.visualization.plot_contour_matrix,
    #pyabc.visualization.plot_acceptance_rates_trajectory,
]:
    abc_plot = plot_func(history)
    plt.tight_layout()


In [86]:
distribution_dfs = []
for t in range(history.max_t + 1):
    df, w = history.get_distribution(m=0, t=t)
    df["t"] = t
    df["w"] = w
    distribution_dfs.append(df)

distribution_dfs = pd.concat(distribution_dfs)
distribution_dfs[["mu_intercept", "mu_slope", "sigma_intercept", "sigma_slope", "t"]].sort_values("t", ascending=False)


name,mu_intercept,mu_slope,sigma_intercept,sigma_slope,t
id,,,,,
542,1.690473,0.062331,0.368331,0.378831,5
543,1.375826,-0.053282,0.258388,0.388080,5
544,1.617884,0.089237,0.432911,0.436799,5
545,1.511748,0.047501,0.413424,0.370457,5
546,1.746502,0.075422,0.405100,0.386678,5
...,...,...,...,...,...
21,-0.137920,0.023270,0.739220,0.161121,0
22,-0.084896,0.057374,0.451154,0.145630,0
23,-1.281830,0.139645,0.029992,0.226772,0


In [82]:
# True hyperparameters
hyper_mu_intercept = 2.0    # Population-level intercept
hyper_sigma_intercept = 0.5 # Between-audience intercept variability
hyper_mu_slope = -0.03      # Population-level cost slope 
hyper_sigma_slope = 0.01    # Between-audience slope variability